# OpenAIでRedisをベクターデータベースとして使用する

このノートブックでは、OpenAI埋め込みを使用してRedisをベクターデータベースとして使用する方法を紹介します。Redisはスケーラブルでリアルタイムなデータベースであり、[RediSearchモジュール](https://oss.redislabs.com/redisearch/)を使用することでベクターデータベースとして利用できます。RediSearchモジュールを使用すると、Redisでベクターのインデックス作成と検索が可能になります。このノートブックでは、RediSearchモジュールを使用して、OpenAI APIで作成しRedisに保存されたベクターのインデックス作成と検索を行う方法を説明します。

### Redisとは？

Webサービス開発の経験がある開発者の多くは、おそらくRedisに馴染みがあるでしょう。Redisの核心は、キャッシュ、メッセージブローカー、データベースとして使用できるオープンソースのキー・バリューストアです。開発者がRedisを選ぶ理由は、高速であること、クライアントライブラリの豊富なエコシステムがあること、そして長年にわたって大企業で導入されてきた実績があることです。

Redisの従来の用途に加えて、Redisは[Redisモジュール](https://redis.io/modules)も提供しており、これは新しいデータ型とコマンドでRedisを拡張する方法です。モジュールの例には、[RedisJSON](https://redis.io/docs/stack/json/)、[RedisTimeSeries](https://redis.io/docs/stack/timeseries/)、[RedisBloom](https://redis.io/docs/stack/bloom/)、[RediSearch](https://redis.io/docs/stack/search/)があります。

### RediSearchとは？

RediSearchは、Redisにクエリ機能、セカンダリインデックス、全文検索、ベクター検索を提供する[Redisモジュール](https://redis.io/modules)です。RediSearchを使用するには、まずRedisデータにインデックスを宣言します。その後、RediSearchクライアントを使用してそのデータをクエリできます。RediSearchの機能セットの詳細については、[README](./README.md)または[RediSearchドキュメント](https://redis.io/docs/stack/search/)を参照してください。

### デプロイメントオプション

Redisをデプロイする方法はいくつかあります。ローカル開発では、最も迅速な方法は[Redis Stack dockerコンテナ](https://hub.docker.com/r/redis/redis-stack)を使用することで、ここではこれを使用します。Redis Stackには、組み合わせて使用することで高速なマルチモデルデータストアとクエリエンジンを作成できる多数のRedisモジュールが含まれています。

本番環境での使用では、最も簡単に始められる方法は[Redis Cloud](https://redislabs.com/redis-enterprise-cloud/overview/)サービスを使用することです。Redis Cloudは完全管理型のRedisサービスです。また、[Redis Enterprise](https://redislabs.com/redis-enterprise/overview/)を使用して独自のインフラストラクチャにRedisをデプロイすることもできます。Redis Enterpriseは、kubernetes、オンプレミス、またはクラウドにデプロイできる完全管理型のRedisサービスです。

さらに、すべての主要クラウドプロバイダー（[AWSマーケットプレイス](https://aws.amazon.com/marketplace/pp/prodview-e6y7ork67pjwg?sr=0-2&ref_=beagle&applicationId=AWSMPContessa)、[Googleマーケットプレイス](https://console.cloud.google.com/marketplace/details/redislabs-public/redis-enterprise?pli=1)、または[Azureマーケットプレイス](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/garantiadata.redis_enterprise_1sp_public_preview?tab=Overview)）がマーケットプレイスでRedis Enterpriseを提供しています。

## 前提条件

このプロジェクトを開始する前に、以下の設定が必要です：

* RediSearchを含むRedisデータベースを起動する（redis-stack）
* ライブラリをインストールする
    * [Redis-py](https://github.com/redis/redis-py)
* [OpenAI APIキー](https://beta.openai.com/account/api-keys)を取得する

===========================================================

### Redisの起動

この例をシンプルに保つため、Redis Stack dockerコンテナを使用します。以下のように起動できます：

```bash
$ docker-compose up -d
```

これには、Redisデータベースを管理するための[RedisInsight](https://redis.com/redis-enterprise/redis-insight/) GUIも含まれており、dockerコンテナを起動すると[http://localhost:8001](http://localhost:8001)で表示できます。

これで設定が完了し、準備が整いました！次に、作成したRedisデータベースと通信するためのクライアントをインポートして作成します。

## 必要なパッケージのインストール

Redis-Pyは、Redisと通信するためのPythonクライアントです。これを使用してRedis-stackデータベースと通信します。

In [ ]:
! pip install redis wget pandas openai

## OpenAI API キーの準備

`OpenAI API key`は、クエリデータのベクトル化に使用されます。

OpenAI API キーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、以下のコマンドを使用して環境変数として`OPENAI_API_KEY`に追加してください：

In [ ]:
! export OPENAI_API_KEY="your API key"

In [2]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os
import openai

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## データの読み込み

このセクションでは、すでにベクトルに変換された埋め込みデータを読み込みます。このデータを使用してRedisにインデックスを作成し、類似ベクトルを検索します。

In [3]:
import sys
import numpy as np
import pandas as pd
from typing import List

# use helper function in nbutils.py to download and read the data
# this should take from 5-10 min to run
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
import nbutils

nbutils.download_wikipedia_data()
data = nbutils.read_wikipedia_data()

data.head()

File Downloaded


,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


## Redisに接続する

Redisデータベースが実行されているので、Redis-pyクライアントを使用して接続できます。Redisデータベースのデフォルトのホストとポートである`localhost:6379`を使用します。

In [4]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

## Redisでの検索インデックスの作成

以下のセルでは、Redisで検索インデックスを指定し、作成する方法を示します。以下の手順を行います：

1. 距離メトリックやインデックス名など、インデックスを定義するための定数を設定
2. RediSearchフィールドを使用してインデックススキーマを定義
3. インデックスを作成

In [5]:
# Constants
VECTOR_DIM = len(data['title_vector'][0]) # length of the vectors
VECTOR_NUMBER = len(data)                 # initial number of vectors
INDEX_NAME = "embeddings-index"           # name of the search index
PREFIX = "doc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [6]:
# Define RediSearch fields for each of the columns in the dataset
title = TextField(name="title")
url = TextField(name="url")
text = TextField(name="text")
title_embedding = VectorField("title_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
text_embedding = VectorField("content_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
fields = [title, url, text, title_embedding, text_embedding]

In [7]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
)

## インデックスにドキュメントを読み込む

検索インデックスができたので、ドキュメントを読み込むことができます。前の例で使用したのと同じドキュメントを使用します。Redisでは、HASH または JSON（RediSearchに加えてRedisJSONを使用する場合）データ型を使用してドキュメントを保存できます。この例ではHASHデータ型を使用します。以下のセルでは、インデックスにドキュメントを読み込む方法を示します。

In [8]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for doc in records:
        key = f"{prefix}:{str(doc['id'])}"

        # create byte vectors for title and content
        title_embedding = np.array(doc["title_vector"], dtype=np.float32).tobytes()
        content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # replace list of floats with byte vectors
        doc["title_vector"] = title_embedding
        doc["content_vector"] = content_embedding

        client.hset(key, mapping = doc)

In [9]:
index_documents(redis_client, PREFIX, data)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Loaded 25000 documents in Redis search index with name: embeddings-index


## OpenAI Query Embeddingsを使用したシンプルなベクトル検索クエリ

検索インデックスが作成され、ドキュメントが読み込まれたので、検索クエリを実行できます。以下では、検索クエリを実行して結果を返す関数を提供します。この関数を使用して、Redisをベクトルデータベースとして活用する方法を示すいくつかのクエリを実行します。

In [10]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "embeddings-index",
    vector_field: str = "title_vector",
    return_fields: list = ["title", "url", "text", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
    print_results: bool = True,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                            model="text-embedding-3-small",
                                            )["data"][0]['embedding']

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    if print_results:
        for i, article in enumerate(results.docs):
            score = 1 - float(article.vector_score)
            print(f"{i}. {article.title} (Score: {round(score ,3) })")
    return results.docs

In [11]:
# For using OpenAI to generate query embedding
results = search_redis(redis_client, 'modern art in Europe', k=10)

0. Museum of Modern Art (Score: 0.875)
1. Western Europe (Score: 0.868)
2. Renaissance art (Score: 0.864)
3. Pop art (Score: 0.86)
4. Northern Europe (Score: 0.855)
5. Hellenistic art (Score: 0.853)
6. Modernist literature (Score: 0.847)
7. Art film (Score: 0.843)
8. Central Europe (Score: 0.843)
9. European (Score: 0.841)


In [12]:
results = search_redis(redis_client, 'Famous battles in Scottish history', vector_field='content_vector', k=10)

0. Battle of Bannockburn (Score: 0.869)
1. Wars of Scottish Independence (Score: 0.861)
2. 1651 (Score: 0.853)
3. First War of Scottish Independence (Score: 0.85)
4. Robert I of Scotland (Score: 0.846)
5. 841 (Score: 0.844)
6. 1716 (Score: 0.844)
7. 1314 (Score: 0.837)
8. 1263 (Score: 0.836)
9. William Wallace (Score: 0.835)


## Redisを使用したハイブリッドクエリ

前の例では、RediSearchを使用してベクトル検索クエリを実行する方法を示しました。このセクションでは、ハイブリッド検索のためにベクトル検索を他のRediSearchフィールドと組み合わせる方法を説明します。以下の例では、ベクトル検索と全文検索を組み合わせます。

In [13]:
def create_hybrid_field(field_name: str, value: str) -> str:
    return f'@{field_name}:"{value}"'

# search the content vector for articles about famous battles in Scottish history and only include results with Scottish in the title
results = search_redis(redis_client,
                       "Famous battles in Scottish history",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("title", "Scottish")
                       )

0. First War of Scottish Independence (Score: 0.892)
1. Wars of Scottish Independence (Score: 0.889)
2. Second War of Scottish Independence (Score: 0.879)
3. List of Scottish monarchs (Score: 0.873)
4. Scottish Borders (Score: 0.863)


In [14]:
# run a hybrid query for articles about Art in the title vector and only include results with the phrase "Leonardo da Vinci" in the text
results = search_redis(redis_client,
                       "Art",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("text", "Leonardo da Vinci")
                       )

# find specific mention of Leonardo da Vinci in the text that our full-text-search query returned
mention = [sentence for sentence in results[0].text.split("\n") if "Leonardo da Vinci" in sentence][0]
mention

0. Art (Score: 1.0)
1. Paint (Score: 0.896)
2. Renaissance art (Score: 0.88)
3. Painting (Score: 0.874)
4. Renaissance (Score: 0.846)


'In Europe, after the Middle Ages, there was a "Renaissance" which means "rebirth". People rediscovered science and artists were allowed to paint subjects other than religious subjects. People like Michelangelo and Leonardo da Vinci still painted religious pictures, but they also now could paint mythological pictures too. These artists also invented perspective where things in the distance look smaller in the picture. This was new because in the Middle Ages people would paint all the figures close up and just overlapping each other. These artists used nudity regularly in their art.'

## HNSWインデックス

これまで、クエリを実行するために``FLAT``または「ブルートフォース」インデックスを使用してきました。Redisは``HNSW``インデックスもサポートしており、これは高速で近似的なインデックスです。``HNSW``インデックスは、ベクトルを格納するために階層的ナビゲート可能スモールワールドグラフを使用するグラフベースのインデックスです。``HNSW``インデックスは、近似クエリを実行したい大規模なデータセットに適した選択肢です。

``HNSW``は、ほとんどの場合において``FLAT``よりも構築に時間がかかり、より多くのメモリを消費しますが、特に大規模なデータセットにおいて、クエリの実行はより高速になります。

以下のセルでは、``HNSW``インデックスを作成し、以前と同じデータを使用してクエリを実行する方法を示します。

In [15]:
# re-define RediSearch vector fields to use HNSW index
title_embedding = VectorField("title_vector",
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER
    }
)
text_embedding = VectorField("content_vector",
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER
    }
)
fields = [title, url, text, title_embedding, text_embedding]

In [16]:
import time
# Check if index exists
HNSW_INDEX_NAME = INDEX_NAME+ "_HNSW"

try:
    redis_client.ft(HNSW_INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(HNSW_INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

# since RediSearch creates the index in the background for existing documents, we will wait until
# indexing is complete before running our queries. Although this is not necessary for the first query,
# some queries may take longer to run if the index is not fully built. In general, Redis will perform
# best when adding new documents to existing indices rather than new indices on existing documents.
while redis_client.ft(HNSW_INDEX_NAME).info()["indexing"] == "1":
    time.sleep(5)

In [17]:
results = search_redis(redis_client, 'modern art in Europe', index_name=HNSW_INDEX_NAME, k=10)

0. Western Europe (Score: 0.868)
1. Northern Europe (Score: 0.855)
2. Central Europe (Score: 0.843)
3. European (Score: 0.841)
4. Eastern Europe (Score: 0.839)
5. Europe (Score: 0.839)
6. Western European Union (Score: 0.837)
7. Southern Europe (Score: 0.831)
8. Western civilization (Score: 0.83)
9. Council of Europe (Score: 0.827)


In [18]:
# compare the results of the HNSW index to the FLAT index and time both queries
def time_queries(iterations: int = 10):
    print(" ----- Flat Index ----- ")
    t0 = time.time()
    for i in range(iterations):
        results_flat = search_redis(redis_client, 'modern art in Europe', k=10, print_results=False)
    t0 = (time.time() - t0) / iterations
    results_flat = search_redis(redis_client, 'modern art in Europe', k=10, print_results=True)
    print(f"Flat index query time: {round(t0, 3)} seconds\n")
    time.sleep(1)
    print(" ----- HNSW Index ------ ")
    t1 = time.time()
    for i in range(iterations):
        results_hnsw = search_redis(redis_client, 'modern art in Europe', index_name=HNSW_INDEX_NAME, k=10, print_results=False)
    t1 = (time.time() - t1) / iterations
    results_hnsw = search_redis(redis_client, 'modern art in Europe', index_name=HNSW_INDEX_NAME, k=10, print_results=True)
    print(f"HNSW index query time: {round(t1, 3)} seconds")
    print(" ------------------------ ")
time_queries()

 ----- Flat Index ----- 
0. Museum of Modern Art (Score: 0.875)
1. Western Europe (Score: 0.867)
2. Renaissance art (Score: 0.864)
3. Pop art (Score: 0.861)
4. Northern Europe (Score: 0.855)
5. Hellenistic art (Score: 0.853)
6. Modernist literature (Score: 0.847)
7. Art film (Score: 0.843)
8. Central Europe (Score: 0.843)
9. Art (Score: 0.842)
Flat index query time: 0.263 seconds

 ----- HNSW Index ------ 
0. Western Europe (Score: 0.867)
1. Northern Europe (Score: 0.855)
2. Central Europe (Score: 0.843)
3. European (Score: 0.841)
4. Eastern Europe (Score: 0.839)
5. Europe (Score: 0.839)
6. Western European Union (Score: 0.837)
7. Southern Europe (Score: 0.831)
8. Western civilization (Score: 0.83)
9. Council of Europe (Score: 0.827)
HNSW index query time: 0.129 seconds
 ------------------------ 
